In [14]:

import numpy as np
import pandas as pd 
from pathlib import Path
from typing import Any, Callable, List, Tuple
import json
import multiprocessing as mp
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import pytorch_lightning as pl

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import io, models, transforms



device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

%matplotlib inline
print(torch.__version__, pl.__version__)

2.0.0 2.0.1.post0


In [15]:
ROOT_PATH = Path("/kaggle/input/cassava-leaf-disease-classification/")
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
LR = 1e-3
EPOCHS = 3

labels = {
    0:"Cassava Bacterial Blight (CBB)",
    1:"Cassava Brown Streak Disease (CBSD)",
    2:"Cassava Green Mottle (CGM)",
    3:"Cassava Mosaic Disease (CMD)",
    4:"Healthy",
}

In [16]:
df = pd.read_csv(ROOT_PATH / "train.csv")
df.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [17]:


train_df, valid_df = train_test_split(df, stratify=df["label"].values)
    
plt.figure(figsize=(12, 5))
print(df["label"].map(labels).value_counts())
df.sample(5)

Cassava Mosaic Disease (CMD)           13158
Healthy                                 2577
Cassava Green Mottle (CGM)              2386
Cassava Brown Streak Disease (CBSD)     2189
Cassava Bacterial Blight (CBB)          1087
Name: label, dtype: int64


,image_id,label
12150,3176028684.jpg,3
18836,518284284.jpg,3
9571,269761139.jpg,3
9079,2604878350.jpg,4
13425,340110323.jpg,4


<Figure size 1200x500 with 0 Axes>

In [18]:
valid_df

,image_id,label
8012,2423344068.jpg,0
11953,3140362576.jpg,3
19104,56813625.jpg,3
4291,1766960814.jpg,3
2874,1497417417.jpg,1
...,...,...
17948,4231168569.jpg,3
13333,3384693576.jpg,3
20052,737753256.jpg,3
14322,3561279698.jpg,3


In [19]:
class Data(Dataset):
    def __init__(self, df: pd.DataFrame, transforms=None):
        self.files = [ROOT_PATH / "train_images" / file for file in df["image_id"].values]
        self.y = df["label"].values.tolist()
        self.transforms = transforms
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, i):
        img = Image.open(self.files[i])
        label = self.y[i]
        if self.transforms is not None:
            img = self.transforms(img)
            
        return img, label

In [25]:
train_tfms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.RandomHorizontalFlip(),\
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize([0.4766, 0.4527, 0.3926], [0.2275, 0.2224, 0.2210])
    ]
)

valid_tfms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.4766, 0.4527, 0.3926], [0.2275, 0.2224, 0.2210])
    ]
)

train_ds = Data(train_df, train_tfms)
valid_ds = Data(valid_df, valid_tfms)

train_dl = DataLoader(
    train_ds,
    BATCH_SIZE, 
    shuffle=True, 
    drop_last=True, 
    num_workers=4,
    pin_memory=True,
)

valid_dl = DataLoader(
    valid_ds, 
    BATCH_SIZE*2, 
    shuffle=False, 
    drop_last=False, 
    num_workers=4,
    pin_memory=True,
)

In [26]:
x, y = next(iter(train_dl))
x.shape, y.shape

(torch.Size([32, 3, 128, 128]), torch.Size([32]))

In [27]:
label_counts = train_df["label"].value_counts().sort_index()
class_weights = max(label_counts) / label_counts.values
label_counts, class_weights

(0     815
 1    1642
 2    1789
 3    9868
 4    1933
 Name: label, dtype: int64,
 array([12.10797546,  6.00974421,  5.51593069,  1.        ,  5.10501811]))

In [28]:
class MyModel(pl.LightningModule):
    def __init__(self, num_classes=5, learning_rate=0.001):
        super().__init__()
        self.learning_rate = learning_rate
        self.num_classes = num_classes
        
   
        self.resnet = models.resnet50(pretrained=True)
        
        # Resize the input layer to match the image size and RGB channels
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Freeze the weights of the ResNet-50 model
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.resnet.fc.out_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
        
        # Loss function
        self.criterion = nn.CrossEntropyLoss()
    
    def forward(self, x):
        features = self.resnet(x)
        features = torch.flatten(features, 1)  # Flatten the tensor
        x = self.fc(features)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)
        return loss




In [29]:
model = MyModel()
#model = model.to(device)
trainer = pl.Trainer(precision=16, max_epochs=10, devices = 1, accelerator='gpu')
trainer.fit(model, train_dl, valid_dl)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/conda/lib/python3.10/site-packages/lightning_fabric/connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]